In [102]:
import cvxpy as cp
import numpy as np
import pandas as pd
import itertools
import statsmodels.api as sm
import warnings
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import AdaBoostRegressor
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [6]:
home = raw_input("Which Home?")
file_path = 'Year1/Home' + home + '_yr1.csv'
print(file_path)

Which Home?1
Year1/Home1_yr1.csv


In [7]:
#print(file_path)
dft = pd.read_csv(file_path, names=['Actual'])
#print(dft.head())
intervals = pd.date_range(start='12/1/2014', end='12/1/2015', freq='15min', closed=None)[0:-1]
dft.insert(0, 'Timestamp', intervals)
dft = dft.set_index('Timestamp')

#print(dft.index)
#print(dft['2014-12-1 00:00:00':])

In [8]:
year = raw_input("Enter the Year:")
month = raw_input("Enter the Month:")
date = raw_input("Enter the Date:")
hour = raw_input("Enter the Hour:")
minute = raw_input("Enter the Minute:")
date1 = datetime(int(year), int(month), int(date), int(hour), int(minute) , 0)
print(date1)

Enter the Year:2015
Enter the Month:11
Enter the Date:1
Enter the Hour:0
Enter the Minute:0
2015-11-01 00:00:00


In [58]:
# Find the end interval after 7 days
print(date1)
end_dt = date1 + timedelta(days=7) - timedelta(minutes=15)
print(end_dt)

training_dataset = dft[: date1 - timedelta(minutes=15)]
dataset = dft[date1:end_dt]
type(dataset['Actual'])

print(dataset['Actual'])

2015-11-01 00:00:00
2015-11-07 23:45:00
Timestamp
2015-11-01 00:00:00    1.16200
2015-11-01 00:15:00    1.27070
2015-11-01 00:30:00    0.65210
2015-11-01 00:45:00    1.22880
2015-11-01 01:00:00    0.83707
2015-11-01 01:15:00    0.99290
2015-11-01 01:30:00    1.06410
2015-11-01 01:45:00    1.01790
2015-11-01 02:00:00    1.27610
2015-11-01 02:15:00    0.57318
2015-11-01 02:30:00    1.15700
2015-11-01 02:45:00    1.32680
2015-11-01 03:00:00    0.57308
2015-11-01 03:15:00    1.35390
2015-11-01 03:30:00    1.31850
2015-11-01 03:45:00    0.71527
2015-11-01 04:00:00    3.07300
2015-11-01 04:15:00    0.90890
2015-11-01 04:30:00    1.16540
2015-11-01 04:45:00    1.34300
2015-11-01 05:00:00    0.52623
2015-11-01 05:15:00    1.37700
2015-11-01 05:30:00    0.57932
2015-11-01 05:45:00    1.29150
2015-11-01 06:00:00    1.28270
2015-11-01 06:15:00    0.57947
2015-11-01 06:30:00    1.43490
2015-11-01 06:45:00    1.20180
2015-11-01 07:00:00    0.70401
2015-11-01 07:15:00    1.22280
                    

In [10]:
#Offline Dynamic 
p = 0.1
a = 1
# use the y as directly from the house values
y = dataset['Actual'].tolist()
x = cp.Variable(672)
sum = 0

for i in range(1,672):
    sum += (cp.multiply(p,x[i]) + cp.maximum(0, y[i]-x[i]) + cp.abs(x[i] - x[i-1]))
    
objective = (cp.Minimize(sum))
constraints = [0 <= x]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
print(result)
# The optimal value for x is stored in `x.value`.
print(x.value)

# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
#print(constraints[0].dual_value)

196.90778135184462
[1.29738135 1.29738108 1.29738025 1.29737888 1.29737702 1.2973747
 1.29737199 1.29736895 1.29736568 1.29736226 1.29735882 1.32684463
 1.32685165 1.35386455 1.35386125 1.3538587  1.39099121 1.39098922
 1.39098701 1.39098456 1.39098182 1.39097875 1.3909753  1.39097145
 1.39096718 1.39096249 1.43509907 1.43514229 1.4351819  1.43521731
 1.43524795 1.43527335 1.97254866 1.97250768 1.9724716  1.97244104
 1.97241655 1.9723986  1.9723875  1.97238351 1.9739609  2.9121294
 2.91194794 2.91171525 2.91143523 2.91111257 2.9107527  2.91036164
 2.90994593 2.90951254 2.90906869 2.90862175 2.90817915 2.9077482
 2.90733599 2.90694929 2.9065944  2.90627709 2.90600248 2.90577498
 2.90559819 2.99184476 2.99187884 2.99190524 2.991924   2.99193522
 2.991939   2.99193545 3.38535471 3.38533148 3.38529598 3.38524849
 3.3851894  3.38511915 3.38503826 3.38494733 3.38484702 3.38473807
 3.38462126 3.3846813  2.62193916 2.62200947 2.62209679 2.62219977
 2.6223168  2.62244601 0.67673379 0.67670013 0

In [12]:
#Offline static

p = 0.1
a = 1
# use the y as directly from the house values
y = dataset['Actual'].tolist()
x = cp.Variable(672)

equation = p*x + a*(cp.maximum(0,y-x))
objective = cp.Minimize(cp.sum(equation))
constraints = [0 <= x]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
print(result)
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
#print(constraints[0].dual_value)

71.14318760000002
[1.162    1.2707   0.6521   1.2288   0.83707  0.9929   1.0641   1.0179
 1.2761   0.57318  1.157    1.3268   0.57308  1.3539   1.3185   0.71527
 3.073    0.9089   1.1654   1.343    0.52623  1.377    0.57932  1.2915
 1.2827   0.57947  1.4349   1.2018   0.70401  1.2228   1.4201   0.66615
 1.9728   0.60645  0.67536  1.2328   1.084    1.868    1.3508   0.77947
 1.9745   3.7043   0.81288  1.2718   1.2348   1.0295   1.4376   0.79793
 1.2804   1.7818   3.9348   1.6702   1.4538   1.5916   1.5587   2.8566
 0.89841  2.3705   0.73691  0.77128  2.1107   2.9912   0.66459  0.43532
 0.34864  0.62942  0.514    0.58552  3.8796   0.70218  2.9555   3.3787
 6.3582   1.6213   0.66265  0.66548  0.55568  0.68982  3.3839   4.6636
 0.61975  0.44291  0.74291  0.51534  0.69445  2.6231   0.57427  0.55883
 0.62702  0.46078  0.6786   0.3853   0.62347  0.62868  0.5798   0.65618
 0.59971  0.58339  0.67095  0.58097  0.60287  0.64961  0.58256  0.60593
 0.64915  0.58033  0.59216  0.73788  0.58209  0.581

In [ ]:
# Implement OGD here !!



In [ ]:
def df(y , prev_x):
    if y > prev_x:
        return 0.4
    else:
        return 0.4 + 4

In [ ]:
# previous OGD - Use x and y as needed similar to offline implementations done above
result = []
for y in y_pred_1:
    print(y)
    cur_x = 3 # The algorithm starts at x=3
    rate = 0.01 # Learning rate
    precision = 0.000001 #This tells us when to stop the algorithm
    previous_step_size = 1 #
    max_iters = 10000 # maximum number of iterations
    iters = 0 #iteration counter
#     df = lambda x: 2*(x+5)
    
    while previous_step_size > precision and iters < max_iters:
        prev_x = cur_x #Store current x value in prev_x
        
        cur_x = cur_x - rate * df(y, prev_x) #Grad descent
        previous_step_size = abs(cur_x - prev_x) #Change in x
        iters = iters+1 #iteration count
        #print("Iteration",iters,"\nX value is",cur_x) #Print iterations
    #print(iters)
    print(cur_x)


In [13]:
# Predictions for RHC and CHC ahead
# ARIMA algorithm starts
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter combinations for Seasonal ARIMA
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [14]:
# find the optimal set of parameters that yields the best performance for our model
# Choose the one with the least resulting AIC value
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(training_dataset,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

ARIMA(0, 0, 0)x(0, 0, 0, 12)12 - AIC:133726.031694
ARIMA(0, 0, 0)x(0, 0, 1, 12)12 - AIC:122330.93617
ARIMA(0, 0, 0)x(0, 1, 0, 12)12 - AIC:118052.747502
ARIMA(0, 0, 0)x(0, 1, 1, 12)12 - AIC:100274.730985
ARIMA(0, 0, 0)x(1, 0, 0, 12)12 - AIC:112693.36413
ARIMA(0, 0, 0)x(1, 0, 1, 12)12 - AIC:100303.768714
ARIMA(0, 0, 0)x(1, 1, 0, 12)12 - AIC:111406.463048
ARIMA(0, 0, 0)x(1, 1, 1, 12)12 - AIC:99913.3606316
ARIMA(0, 0, 1)x(0, 0, 0, 12)12 - AIC:115512.856194
ARIMA(0, 0, 1)x(0, 0, 1, 12)12 - AIC:110207.139274
ARIMA(0, 0, 1)x(0, 1, 0, 12)12 - AIC:113691.698071
ARIMA(0, 0, 1)x(0, 1, 1, 12)12 - AIC:94455.1304988
ARIMA(0, 0, 1)x(1, 0, 0, 12)12 - AIC:105926.570048
ARIMA(0, 0, 1)x(1, 0, 1, 12)12 - AIC:94483.7409564
ARIMA(0, 0, 1)x(1, 1, 0, 12)12 - AIC:106048.098853
ARIMA(0, 0, 1)x(1, 1, 1, 12)12 - AIC:94316.2999333
ARIMA(0, 1, 0)x(0, 0, 0, 12)12 - AIC:103020.588004
ARIMA(0, 1, 0)x(0, 0, 1, 12)12 - AIC:102981.182242
ARIMA(0, 1, 0)x(0, 1, 0, 12)12 - AIC:124955.583046
ARIMA(0, 1, 0)x(0, 1, 1, 12)12 - 

In [15]:
pdq = [1, 1, 0, 1, 1, 1, 1]
print(pdq)

[1, 1, 0, 1, 1, 1, 1]


In [16]:
mod = sm.tsa.statespace.SARIMAX(training_dataset,
                                order=(pdq[1], pdq[2], pdq[3]),
                                seasonal_order=(pdq[4], pdq[5], pdq[6], 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8378      0.004    195.448      0.000       0.829       0.846
ma.L1         -0.5364      0.006    -88.313      0.000      -0.548      -0.525
ar.S.L12       0.0187      0.005      4.129      0.000       0.010       0.028
ma.S.L12      -1.0124      0.001  -1090.441      0.000      -1.014      -1.011
sigma2         0.9994      0.004    227.618      0.000       0.991       1.008


In [97]:
# Forecasting
pred = results.get_prediction(start=date1, end=end_dt+timedelta(hours=1)+timedelta(minutes=15), dynamic=False)
arima_pred_list = pred.predicted_mean.values.tolist()
y_pred_arima = np.asarray(arima_pred_list)
df_forecasted = pred.predicted_mean[:672]
df_truth = dataset['Actual']
print("Mean absolute error LR: %.9f"
      % mean_absolute_error(df_truth,df_forecasted))

Mean absolute error LR: 0.758411392


In [98]:
print(y_pred_arima)
print(len(y_pred_arima))

[1.20483631 1.25149979 1.19972778 1.27670541 1.16253604 1.25523705
 1.31329419 1.40152866 1.2553715  1.16265196 1.12715142 1.08302046
 1.15668318 1.22810432 1.183734   1.2509107  1.15238625 1.23478308
 1.30949076 1.33702725 1.20163835 1.1641847  1.1191451  1.08478132
 1.15138047 1.22397899 1.18034519 1.24783919 1.15002753 1.23258296
 1.30789719 1.33454341 1.19956306 1.16331805 1.11824493 1.08418577
 1.15075451 1.22346047 1.17991202 1.24747191 1.14972382 1.23232431
 1.30768516 1.33434424 1.1993963  1.16319466 1.11813829 1.08409939
 1.15067976 1.22339796 1.17985967 1.24742797 1.14968708 1.23229345
 1.30765938 1.33432224 1.19937787 1.16317953 1.11812555 1.08408877
 1.15067082 1.22339047 1.17985339 1.24742271 1.14968267 1.23228975
 1.30765629 1.33431964 1.19937569 1.16317771 1.11812402 1.0840875
 1.15066975 1.22338957 1.17985264 1.24742208 1.14968215 1.23228931
 1.30765592 1.33431933 1.19937543 1.16317749 1.11812384 1.08408734
 1.15066962 1.22338947 1.17985255 1.247422   1.14968208 1.23228

In [74]:
print(file_path)
df = pd.read_csv(file_path, names=['Consumption'])
#print(df)
def datetime_range(start, end, delta):
    end += 4*delta
    current = start
    while current < end:
        yield current
        current += delta

dts = [dt for dt in 
       datetime_range(datetime(2014, 12, 1, 0), datetime(2015, 11 , 30 , 23), 
       timedelta(minutes=15))]
df['date/time'] = dts

Year1/Home1_yr1.csv


In [75]:
for i, row in df.iterrows():
#handling seasons
    df.at[i, 'Year'] = row[1].year
    df.at[i, 'Date'] = row[1].day
    df.at[i, 'Month'] = row[1].month
    df.at[i, 'Hour'] = row[1].hour
    df.at[i, 'Minute'] = row[1].minute
    if row[1].month in range(1,4) :
        df.at[i, 'Summer'] = 1
    elif row[1].month in range(4,7):
        df.at[i, 'Spring'] = 1
    elif row[1].month in range(7,10):
        df.at[i, 'Fall'] = 1
    elif row[1].month in range(10,13):
        df.at[i, 'Winter'] = 1
        
#handling peak hours   
    if row[1].hour in [0,1,2,3,4,5,6,7,20,21,22,23]:
        df.at[i, 'Peak Hour'] = 1
    else:
        df.at[i, 'Peak Hour'] = 0
        
#handling holidays & weekends
    if row[1].weekday() in [5,6] :
        df.at[i, 'Holiday'] = 1
    elif row[1].month == 10 and row[1].day == 2:
        df.at[i, 'Holiday'] = 1
    elif row[1].month == 1 and row[1].day == 26:
        df.at[i, 'Holiday'] = 1
    else:
        df.at[i, 'Holiday'] = 0

df['Spring'].fillna(0,inplace=True)
df['Fall'].fillna(0,inplace=True)
df['Summer'].fillna(0,inplace=True)
df['Winter'].fillna(0,inplace=True)

In [89]:
for i, row in df.iterrows():
    if row[1] == date1:
        break
x_train = df[['Year','Date','Month','Hour','Minute','Summer','Winter', 'Fall','Spring' , 'Holiday', 'Peak Hour']][:i]
y_train = df[['Consumption']][:i]
x_test = df[['Year','Date','Month','Hour','Minute','Summer','Winter', 'Fall','Spring' , 'Holiday', 'Peak Hour']] [i:677+i]
y_test = df[['Consumption']][i:677+i]

In [77]:
print(x_train)

         Year  Date  Month  Hour  Minute  Summer  Winter  Fall  Spring  \
0      2014.0   1.0   12.0   0.0     0.0     0.0     1.0   0.0     0.0   
1      2014.0   1.0   12.0   0.0    15.0     0.0     1.0   0.0     0.0   
2      2014.0   1.0   12.0   0.0    30.0     0.0     1.0   0.0     0.0   
3      2014.0   1.0   12.0   0.0    45.0     0.0     1.0   0.0     0.0   
4      2014.0   1.0   12.0   1.0     0.0     0.0     1.0   0.0     0.0   
5      2014.0   1.0   12.0   1.0    15.0     0.0     1.0   0.0     0.0   
6      2014.0   1.0   12.0   1.0    30.0     0.0     1.0   0.0     0.0   
7      2014.0   1.0   12.0   1.0    45.0     0.0     1.0   0.0     0.0   
8      2014.0   1.0   12.0   2.0     0.0     0.0     1.0   0.0     0.0   
9      2014.0   1.0   12.0   2.0    15.0     0.0     1.0   0.0     0.0   
10     2014.0   1.0   12.0   2.0    30.0     0.0     1.0   0.0     0.0   
11     2014.0   1.0   12.0   2.0    45.0     0.0     1.0   0.0     0.0   
12     2014.0   1.0   12.0   3.0     0

In [78]:
print(y_train)

       Consumption
0          0.65018
1          1.45400
2          0.72971
3          3.10750
4          0.63572
5          0.69720
6          1.50610
7          0.85858
8          1.61320
9          0.66075
10         0.50952
11         1.53310
12         0.64315
13         1.55420
14         0.63596
15         1.12640
16         0.94811
17         0.94247
18         1.03720
19         1.17440
20         0.87564
21         2.68930
22         0.82416
23         1.33590
24         0.59462
25         1.31450
26         0.62424
27         1.88760
28         1.35300
29         1.97930
...            ...
32130      0.72927
32131      0.73064
32132      0.59854
32133      1.29840
32134      1.86110
32135      0.79159
32136      0.71204
32137      0.71036
32138      0.77162
32139      0.73239
32140      0.71010
32141      0.75897
32142      0.73297
32143      0.70385
32144      2.39010
32145      0.90559
32146      2.20200
32147      0.72889
32148      1.81230
32149      0.85572
32150      0

In [99]:
regr = AdaBoostRegressor(base_estimator=None, n_estimators=50, learning_rate=1.0, loss='linear', random_state=None)
regr.fit(x_train, y_train)
y_pred_adaboost = regr.predict(x_test)
print("Mean absolute error LR: %.9f"
      % mean_absolute_error(y_test,y_pred_adaboost))
print(len(y_pred_adaboost))
print(y_pred_adaboost)

Mean absolute error LR: 1.789415636
677
[2.38658656 2.39645962 2.39645962 2.39645962 2.38658656 2.39645962
 2.39645962 2.39645962 2.38658656 2.39645962 2.39645962 2.39645962
 2.38658656 2.39645962 2.39645962 2.39645962 2.38658656 2.39645962
 2.39645962 2.39645962 2.38658656 2.39645962 2.39645962 2.39645962
 2.38658656 2.39645962 2.39645962 2.39645962 2.38658656 2.49500025
 2.49500025 2.49500025 2.49500025 2.59004365 2.59004365 2.59004365
 2.49500025 2.59004365 2.59004365 2.59004365 2.59004365 2.59004365
 2.59004365 2.59004365 3.04025045 2.98822808 2.98822808 2.98822808
 3.03131939 2.98822808 2.98822808 2.98822808 2.83652042 2.98822808
 2.98822808 2.98822808 2.83652042 2.98822808 2.98822808 2.98822808
 2.83652042 2.98822808 2.98822808 2.98822808 2.83652042 2.98822808
 2.98822808 2.98822808 2.83652042 2.98822808 2.98822808 2.98822808
 2.83652042 2.98822808 2.98822808 2.98822808 2.83652042 2.98822808
 2.98822808 2.98822808 2.83652042 2.98822808 2.98822808 2.98822808
 2.83652042 2.98822808

In [100]:
# RHC with Arima predictions
p = 0.1
a = 1
w = 6
X_1 = []

for t in range(0,672):
    y = y_pred_arima[t:t+w]
    x = cp.Variable(w)
    sum = 0
    for i in range(0,w):
        sum += (cp.multiply(p,x[i]) + cp.maximum(0, y[i]-x[i]) + cp.abs(x[i] - x[i-1]))
    
    objective = (cp.Minimize(sum))
    constraints = [0 <= x]
    prob = cp.Problem(objective, constraints)

    # The optimal objective value is returned by `prob.solve()`.
    result = prob.solve()
    #print(result)
    # The optimal value for x is stored in `x.value`.
    X_1.insert(t, x.value[0])
    
    # The optimal Lagrange multiplier for a constraint is stored in
    # `constraint.dual_value`.
    #print(constraints[0].dual_value)
print(len(X_1))
print(X_1)

672
[1.276705413416717, 1.3132941933760018, 1.4015286636264461, 1.4015286636264461, 1.4015286636264461, 1.4015286636264461, 1.4015286636264461, 1.4015286636264461, 1.255371502860931, 1.228104318920777, 1.2509106977251465, 1.2509106977251465, 1.2509106977251465, 1.309490756468137, 1.337027253383066, 1.337027253383066, 1.337027253383066, 1.337027253383066, 1.337027253383066, 1.337027253383066, 1.2239789870177626, 1.2239789870177626, 1.2478391929296655, 1.2478391929296655, 1.2478391929296655, 1.307897186511611, 1.334543406693727, 1.334543406693727, 1.334543406693727, 1.334543406693727, 1.334543406693727, 1.334543406693727, 1.2234604748677933, 1.2234604748677933, 1.2474719136186256, 1.2474719136186256, 1.2474719136186256, 1.307685156913581, 1.334344238601256, 1.334344238601256, 1.334344238601256, 1.334344238601256, 1.334344238601256, 1.334344238601256, 1.2233979593024347, 1.2233979593024347, 1.2474279667071653, 1.2474279667071653, 1.2474279667071653, 1.3076593837961348, 1.334322241489282, 

In [101]:
# RHC with AdaBoost predictions
p = 0.1
a = 1
w = 6
X_2 = []

for t in range(0,672):
    y = y_pred_adaboost[t:t+w]
    x = cp.Variable(w)
    sum = 0
    for i in range(0,w):
        sum += (cp.multiply(p,x[i]) + cp.maximum(0, y[i]-x[i]) + cp.abs(x[i] - x[i-1]))
    
    objective = (cp.Minimize(sum))
    constraints = [0 <= x]
    prob = cp.Problem(objective, constraints)

    # The optimal objective value is returned by `prob.solve()`.
    result = prob.solve()
    #print(result)
    # The optimal value for x is stored in `x.value`.
    X_2.insert(t, x.value[0])
    
    # The optimal Lagrange multiplier for a constraint is stored in
    # `constraint.dual_value`.
    #print(constraints[0].dual_value)
print(len(X_2))
print(X_2)

672
[2.39264115456497, 2.396459624344704, 2.396459624344704, 2.393078964570009, 2.392641154622756, 2.396459624344704, 2.396459624344704, 2.3930789646202553, 2.392641154541253, 2.396459624344704, 2.396459624344704, 2.3930789646222843, 2.392641154622756, 2.396459624344704, 2.396459624344704, 2.39307896457001, 2.392641154564971, 2.396459624344704, 2.396459624344704, 2.393078964570008, 2.392641154605572, 2.396459624344704, 2.396459624344704, 2.393078964575124, 2.495000249060134, 2.495000249060134, 2.495000249060134, 2.495000249060134, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 2.5900436468094155, 3.0402504467646088, 3.0402504467646088, 3.0402504467646088, 3.0402504467646088, 3.0402504467646088, 3.0402504467646088, 3.0313193901745845, 3.0313193901745845, 3.0313193901745845, 3.0313193901745845, 2.9882280757911404, 2.9882280757911404, 2.9